In [2]:
import numpy as np

import os.path as op

import pandas as pd

import mne
from mne import setup_volume_source_space
from mne.beamformer import apply_lcmv_cov, make_lcmv
from mne.coreg import Coregistration
from mne.cov import compute_covariance
from mne.datasets import somato

from pathlib import Path

import ipywidgets

from scipy.spatial import KDTree



In [3]:
raw_paths = {#"Patient03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/FM_patient3/FM_9to16_part1_raw_tsss.fif",
            'patient_05_part1L' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif" ,
            'patient_05_part1R' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Rhippo_raw_001_tsss.fif",
            'patient_05_part2L' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Lhippo_raw_002_tsss.fif",
            'patient_05_part2R' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_RHippo_raw_002_tsss.fif",
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZM_patient6/ZM_raw_001_tsss.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/XX_patient8/XX_raw_001_tsss.fif",
           # "patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/TY_patient9/TY_raw_001_tsss.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/NY_patient10/NY_raw_001_tsss.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZJ_patient11/ZJ_raw_001_tsss.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZJ_patient11/ZJ_raw_002_tsss.fif",
            #"patient_12" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/JC_patient12/JC_raw_001_tsss.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZB_patient13/ZB_raw_004_tsss.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_001_tsss.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_003_tsss.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_001_tsss.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_003_tsss.fif",
            #"patient_15" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZX_patient15/ZX_EP_rest_001_tsss.fif",
            "patient_16" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/RB_patient16/RB_EP_SEEG_rest_001_tsss.fif",
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/LJM_patient17/LJM_EP_SEEG_rest_001_tsss.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/HH_patient21/HH_SEEG_001_tsss.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/YY_patient22/YY_SEEG_raw_001_tsss.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZQ_patient23/ZQ_EP_SEEG_001_tsss.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WL_patient25/WL_raw_rest_001_tsss.fif"
            }

In [4]:
coregs_files = {"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_03-trans.fif",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_06-trans.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_08-trans.fif",
            "patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_09-trans.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_10-trans.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_11-trans.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_11-trans.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_13-trans.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_16" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_16-trans.fif",
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_17-trans.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_21-trans.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_22-trans.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_23-trans.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_25-trans.fif"
            }

In [5]:
#dictionary for epochsa
all_epochs_files = {#"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient3_part1_all_SWR_epochs.fi",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_all_theta_epochs.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_all_theta_epochs.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2L_all_theta_epochs.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2R_all_theta_epochs.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient6_all_theta_epochs.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_all_theta_epochs.fif",
            #"patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient9_all_SWR_epochs.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_all_theta_epochs.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part1_all_theta_epochs.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part2_all_theta_epochs.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient13_all_theta_epochs.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_left_all_theta_epochs.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_left_all_theta_epochs.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_right_all_theta_epochs.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_right_all_theta_epochs.fif",
            "patient_16" : '/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient16_all_theta_epochs.fif',
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient17_all_theta_epochs.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient21_all_theta_epochs.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient22_all_theta_epochs.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient23_all_theta_epochs.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient25_all_theta_epochs.fif"
            }


high_epochs_files = {#"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient3_part1_high_SWR_epochs.fi",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_high_theta_epochs.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_high_theta_epochs.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2L_high_theta_epochs.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2R_high_theta_epochs.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient6_high_theta_epochs.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_high_theta_epochs.fif",
            #"patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient9_high_SWR_epochs.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_high_theta_epochs.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part1_high_theta_epochs.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part2_high_theta_epochs.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient13_high_theta_epochs.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_left_high_theta_epochs.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_left_high_theta_epochs.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_right_high_theta_epochs.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_right_high_theta_epochs.fif",
            "patient_16" : '/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient16_high_theta_epochs.fif',
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient17_high_theta_epochs.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient21_high_theta_epochs.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient22_high_theta_epochs.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient23_high_theta_epochs.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient25_high_theta_epochs.fif"
            }

low_epochs_files = {#"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient3_part1_low_SWR_epochs.fi",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_low_theta_epochs.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_low_theta_epochs.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2L_low_theta_epochs.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2R_low_theta_epochs.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient6_low_theta_epochs.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_low_theta_epochs.fif",
            "patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient9_low_theta_epochs.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_low_theta_epochs.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part1_low_theta_epochs.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part2_low_theta_epochs.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient13_low_theta_epochs.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_left_low_theta_epochs.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_left_low_theta_epochs.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_right_low_theta_epochs.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_right_low_theta_epochs.fif",
            "patient_16" : '/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient16_low_theta_epochs.fif',
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient17_low_theta_epochs.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient21_low_theta_epochs.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient22_low_theta_epochs.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient23_low_theta_epochs.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient25_low_theta_epochs.fif"
            }

In [6]:
electrode_coords = {
        "patient_05_part1L" : {'EEG026': [-31.9481,  -17.6862,  -21.6874],
                               'EEG028':[-32.1601,  -20.9514, -19.5944],
                               'EEG032':[-32.4263,  -24.2073,  -17.4454],
                               'EEG034':[-32.6851,  -27.4295,  -15.2116],
                               'EEG057':[-32.8854,  -30.5981,  -12.8743],
                               'EEG059':[-33.0629,  -33.8199,  -10.6193]
                               },
        "patient_05_part1R" : {'EEG001':[27.8007,   -7.4679,  -26.6730], 
                               'EEG003':[28.5425,  -10.6786,  -24.4242], 
                               'EEG026':[29.3337,  -13.8591,  -22.2597], 
                               'EEG028':[30.1206,  -16.9794,  -20.1042], 
                               'EEG032':[30.8624,  -20.0294,  -17.9008], 
                               'EEG034': [31.5477,  -23.0244,  -15.6570]
                               },
        "patient_05_part2L" : {'EEG026': [-31.9481,  -17.6862,  -21.6874],
                               'EEG028':[-32.1601,  -20.9514, -19.5944],
                               'EEG032':[-32.4263,  -24.2073,  -17.4454],
                               'EEG034':[-32.6851,  -27.4295,  -15.2116],
                               'EEG057':[-32.8854,  -30.5981,  -12.8743],
                               'EEG059':[-33.0629,  -33.8199,  -10.6193]
                               },
        "patient_05_part2R" : {'EEG001':[27.8007,   -7.4679,  -26.6730], 
                               'EEG003':[28.5425,  -10.6786,  -24.4242], 
                               'EEG026':[29.3337,  -13.8591,  -22.2597], 
                               'EEG028':[30.1206,  -16.9794,  -20.1042], 
                               'EEG032':[30.8624,  -20.0294,  -17.9008], 
                               'EEG034': [31.5477,  -23.0244,  -15.6570]
                               },
        "patient_06" : {'EEG001':[35.0082 ,  -4.1612,  -22.2920], 
                        'EEG002':[34.8448 ,  -7.6846 , -20.2725],
                        'EEG003':[34.8133 , -11.2058 , -18.1837],
                        'EEG004':[35.0223 , -14.8053 , -15.9063],
                        'EEG005':[35.2409 , -18.5894 , -13.6695],
                        'EEG006':[35.7081 , -22.5214 , -11.7750],
                        'EEG007':[35.9444 , -26.4630,  -10.0459],
                        'EEG008':[35.9492,  -30.0867,   -8.0074]
                               },
        "patient_08" : {'EEG026': [-28.7497 ,  -6.3273 , -17.6884],
                        'EEG027': [-29.6078  , -9.9060 , -15.4222],
                        'EEG028':[-30.4463 , -13.6230,  -13.2690]
                               },
        "patient_10" : {'EEG013': [39.0578, -27.7104, -14.8564]
                        },
        "patient_11_part1" : {'EEG043':[25.5465, -6.3252, -26.9552],
                              'EEG044':[29.0527, -6.4683, -26.3527],
                              'EEG045':[32.5691, -6.6747, -25.8155],
                              'EEG046':[35.9451, -6.7906, -25.1166]
                                      },
        "patient_11_part2" : {'EEG043':[25.5465, -6.3252, -26.9552],
                              'EEG044':[29.0527, -6.4683, -26.3527],
                              'EEG045':[32.5691, -6.6747, -25.8155],
                              'EEG046':[35.9451, -6.7906, -25.1166]
                                      },
        "patient_13" : {'EEG036' :[40.7319, -21.8473, -17.7705]
                        },
        "patient_14_part1_left" : {'EEG001':[-34.6168, -28.4119, -13.6643],
                                      },
        "patient_14_part2_left" : {'EEG001':[-34.6168, -28.4119, -13.6643],
                                      },
        "patient_14_part1_right" : {
                              'EEG041':[30.6281, -25.9213, -12.7094],
                              'EEG042':[33.7950, -25.5985, -10.4567],
                              'EEG043':[37.0594, -25.2660,  -8.4478]
                                      },
        "patient_14_part2_right" : {
                              'EEG041':[30.6281, -25.9213, -12.7094],
                              'EEG042':[33.7950, -25.5985, -10.4567],
                              'EEG043':[37.0594, -25.2660,  -8.4478]
                                      },
        "patient_16" : {'EEG003': [-27.2459,  -7.5079, -15.7188]
                        },
        "patient_17" : {'EEG003':[26.6824, -16.3633, -20.1338],
                        'EEG004':[29.9791, -16.5218, -18.0723],
                        'EEG005':[33.2275, -16.5570, -16.0582],
                        'EEG006':[36.5064, -16.6121, -14.2557]
                        },
        "patient_21" : {'EEG002':[-26.5338, -17.4555, -14.5455],
                        'EEG003':[-29.7603, -18.8401, -13.8878],
                        'EEG004':[-32.8542, -20.1940, -13.2209],
                        'EEG005':[-35.9057, -21.5125, -12.5974]
                        },
        "patient_22" : {'EEG001':[-28.1512, -13.4967, -19.3708], 
                        'EEG002':[-32.2938, -12.6457, -18.4760],
                        'EEG003':[-36.2198, -12.1852, -17.7407], 
                        'EEG017':[-21.4524, -22.3667, -12.7231], 
                        'EEG018': [-24.3698, -24.8088, -11.7376], 
                        'EEG019':[-27.2600, -27.4537, -10.9368],
                        'EEG020':[-30.1337, -30.1438, -10.1958],
                        'EEG021':[-32.9257, -32.7396,  -9.3085], 
                        'EEG022':[-35.6350, -35.4950,  -8.5632]
                        },
        "patient_23" : {'EEG057':[29.1695, -16.6921, -23.0606], 
                        'EEG058':[33.2076, -16.3588, -22.6841],
                        'EEG059':[37.4743, -16.4895, -22.9268]
                        },
        "patient_25" : {'EEG049':[-26.6822,  -4.7599, -25.5903],
                        'EEG050':[-28.4079,  -8.2010, -25.1680]
                        }
}



In [7]:
for subject, path in raw_paths.items():
    mne.set_log_level('ERROR')
    raw_fname = op.abspath( raw_paths[subject])
    raw = mne.io.read_raw_fif(raw_fname, verbose = False)
    info = raw.info

    base_subject = subject.split("_part")[0] 
    print(base_subject)
  
    
    data_path = op.abspath("/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS")
    subjects_dir = op.join(data_path, "freesurfer", "recons")
    coreg_fname = op.abspath(coregs_files[subject])
    coreg = mne.read_trans(coreg_fname, verbose = False)

    #improt 3 sets of epochs
    all_epochs_fname = op.abspath(all_epochs_files[subject])
    all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)

    high_epochs_fname = op.abspath(high_epochs_files[subject])
    high_epochs = mne.read_epochs(high_epochs_fname, verbose = False)

    low_epochs_fname = op.abspath(low_epochs_files[subject])
    low_epochs = mne.read_epochs(low_epochs_fname, verbose = False)

    #filter in the ripple band
    all_epochs = all_epochs.filter(2, 5, verbose = False )
    high_epochs = high_epochs.filter(2, 5, verbose = False)
    low_epochs = low_epochs.filter(2, 5, verbose = False)

    #prepare source space
    surface = op.join(subjects_dir, base_subject , "bem" , "inner_skull.surf")
    src = mne.setup_volume_source_space(
    base_subject, subjects_dir=subjects_dir, pos=10.0, surface=surface, add_interpolator=False, verbose = False
    )  

    conductivity = (0.3,)
    model = mne.make_bem_model(
    subject=base_subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir, verbose = False
    )
    bem = mne.make_bem_solution(model, verbose = False)

    #compute forward solution 
    fwd = mne.make_forward_solution(
        raw_fname,
        trans=coreg,
        src=src,
        bem= bem,
        meg=True,
        eeg=False,
        mindist=5.0,
        n_jobs=None,
        verbose=False,
        )
    #prepare covariance matrix
    rank = mne.compute_rank(all_epochs, tol=1e-6, tol_kind="relative", verbose = False)
    active_win = (0, 0.3)
    baseline_win = (0, 0.3)
    common_win = (0, 0.3)

    #epochs of no SWRs
    baseline_cov = compute_covariance(
        low_epochs,
        tmin=baseline_win[0],
        tmax=baseline_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )
    #epochs of SWRs
    active_cov = compute_covariance(
        high_epochs,
        tmin=active_win[0],
        tmax=active_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )

    #epochs of both
    cov = compute_covariance(
        all_epochs,
        tmin=common_win[0],
        tmax=common_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )

    noise_cov = mne.make_ad_hoc_cov(all_epochs.info, verbose = False) 

    
    

    filters = make_lcmv(
        all_epochs.info, fwd, cov,
        reg=0.05, noise_cov = noise_cov, 
        pick_ori="max-power", 
        weight_norm= None, 
        verbose = False
    )

    #electrdoe coords
    coords_dict = electrode_coords[subject]
    elec_labels = list(coords_dict.keys())
    elec_array = np.array([coords_dict[name] for name in elec_labels]) #in MNI space

    trans_file = op.join(data_path, "coregs", base_subject + '-trans.fif')
    mri_head_t = mne.read_trans(trans_file, verbose=False)

    #get the used coords
    used_vertices = filters['vertices'][0]
    head_coords = src[0]['rr'][used_vertices] #this are in meters

    #covert to MNI
    mni_coords = mne.head_to_mni(head_coords, base_subject, mri_head_t, subjects_dir) 

    # Use KDTree search to find nearest neighbours
    tree = KDTree(mni_coords)
    dist,idx_in_tree = tree.query(elec_array)

    matched_vertices = used_vertices[idx_in_tree]
    filters_for_electrodes = filters['weights'][idx_in_tree]

    print(f"Mapped {len(matched_vertices)} electrodes for {subject}:")
    for label, idx, d in zip(elec_labels, matched_vertices, dist):
        print(f"  {label} → vertex {idx} (distance: {d:.1f} mm)")
    
    rows = []
    for label, idx, d, coord in zip(elec_labels, matched_vertices, dist, mni_coords[idx_in_tree]):
        rows.append({
            "subject": subject,
            "channel": label,
            "vertex_index": idx,
            "distance_mm": round(d, 2),
            "vertex_coords_mni": coord.tolist()
        })

    # After the loop for all subjects
    results_df = pd.DataFrame(rows)
    results_df.to_csv(f"/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/virtual_electrode/theta/theta_{subject}_voxel_coords.csv", index=False)



    




/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:4: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_all_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)


patient_05


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:21: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_high_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  high_epochs = mne.read_epochs(high_epochs_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:24: RuntimeWarning: This filename 

Mapped 6 electrodes for patient_05_part1L:
  EEG026 → vertex 3405 (distance: 5.8 mm)
  EEG028 → vertex 3405 (distance: 5.8 mm)
  EEG032 → vertex 3728 (distance: 6.6 mm)
  EEG034 → vertex 3728 (distance: 6.1 mm)
  EEG057 → vertex 3711 (distance: 5.9 mm)
  EEG059 → vertex 3711 (distance: 6.1 mm)
patient_05


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:4: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Rhippo_raw_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_all_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: The events passed to

...

Mapped 8 electrodes for patient_06:
  EEG001 → vertex 2646 (distance: 6.6 mm)
  EEG002 → vertex 2646 (distance: 5.5 mm)
  EEG003 → vertex 2629 (distance: 5.8 mm)
  EEG004 → vertex 2629 (distance: 5.1 mm)
  EEG005 → vertex 2935 (distance: 7.0 mm)
  EEG006 → vertex 2935 (distance: 6.2 mm)
  EEG007 → vertex 2918 (distance: 6.2 mm)
  EEG008 → vertex 2918 (distance: 5.3 mm)
patient_08


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:4: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/XX_patient8/XX_raw_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_all_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: The events passed to the Epochs co

Mapped 3 electrodes for patient_08:
  EEG026 → vertex 2946 (distance: 7.3 mm)
  EEG027 → vertex 2946 (distance: 3.5 mm)
  EEG028 → vertex 2946 (distance: 3.1 mm)
patient_10


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:4: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/NY_patient10/NY_raw_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_all_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57757/1559247986.py:18: RuntimeWarning: The events passed to the Epochs 

KeyboardInterrupt: 